Removing any peyer's patches that passed through accidentally

In [1]:
import scanpy as sc
import numpy as np
import scanpy as sc
import glob
import os
import json
from PIL import Image, ImageDraw
import shapely

In [2]:
output_folder = r"/mnt/sata1/Analysis_Alex/uninfected"

In [ ]:
adata = sc.read(os.path.join(output_folder, "final_uninfected_adata.h5ad"))

In [ ]:
sc.pl.embedding(adata, basis='spatial', color = 'Immunocentric_Type')

Save images of spatial cells colored by Immunocentric cell type

In [ ]:
for input_file in np.unique(adata.obs.batch):
    current_adata = adata[adata.obs['batch'] == input_file]
    all_spatial = current_adata.obsm['X_spatial']

    Immunocentric_Type_categories = current_adata.obs['Immunocentric_Type'].cat.categories
    Immunocentric_Type_colors = current_adata.uns['Immunocentric_Type_colors']
    
    Immunocentric_Type_dict = {category: Immunocentric_Type_colors[i] for i, category in enumerate(Immunocentric_Type_categories)}
    image_colors = [Immunocentric_Type_dict.get(v) for v in current_adata.obs.Immunocentric_Type]

    #D6, D7, D30 #downsize = 20
    downsize = 5
    # Example 2D point array (replace this with your own data)
    points = all_spatial/downsize
    
    # Define the size of the image (adjust as needed)
    image_width = 2000
    image_height = 2000
    
    # Create a white canvas as the base image
    base_image = Image.new('RGB', (image_width, image_height), (255, 255, 255))
    
    # Draw the points on the image
    draw = ImageDraw.Draw(base_image)
    point_size = 1  # Size of the points
    ct = 0
    for point in points:
        draw.ellipse((point[0] - point_size, point[1] - point_size, point[0] + point_size, point[1] + point_size),
                     fill=image_colors[ct])
        ct += 1

    try:
        os.mkdir(os.path.join(output_folder, 'peyers'))
    except:
        print('peyers directory already exists')
        
    try:
        os.mkdir(os.path.join(output_folder, 'peyers', input_file))
    except:
        print('directory already exists')
    file_path = os.path.join(output_folder, 'peyers', input_file, 'Immunocentric_Type_image.png')
    base_image.save(file_path)

label peyers patch cells to remove in labelme

parse these labels below

In [ ]:
from tqdm.notebook import tqdm
from shapely.geometry import Point, Polygon

remove_ids = []
for input_file in tqdm(np.unique(adata.obs.batch)):
    current_adata = adata[adata.obs['batch'] == input_file]
    json_file_path =os.path.join(output_folder, 'peyers', input_file, 'Immunocentric_Type_image.json')
    all_spatial = current_adata.obsm['X_spatial']
    
    # Load the JSON data from the file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    
    # Extract relevant information from the JSON data
    image_height = data['imageHeight']
    image_width = data['imageWidth']
    image_path = data['imagePath']
    shapes = data['shapes']
    
    # Process the shapes (annotations)
    removals = []
    points = [] 
    top_points = [] 
    for shape in shapes:
        label = shape['label']
        removals.append(shape['points'])
    
    total_indices = []
    for ir in removals:
        ir_ = np.array(ir)*downsize
        poly = Polygon(ir_)
        indices = []
        for i in range(len(all_spatial)):
            pt = Point(all_spatial[i])
            if pt.within(poly):
                indices.append(i)
        total_indices.append(indices)
    
    total_indices=list(set([element for sublist in total_indices for element in sublist]))
    index_set = list(set(total_indices))
    if len(index_set) > 0:
        current_adata = current_adata[index_set, :]
        remove_ids.extend(current_adata.obs.index.values)
    print(len(remove_ids))

In [7]:
adata = adata[~adata.obs.index.isin(remove_ids)]

In [8]:
adata.write(os.path.join(output_folder, 'analysis/cleaned/final_object_no_peyers.h5ad'))

Converting P14s to cd8ab

In [ ]:
adata = sc.read(os.path.join(output_folder, 'analysis/cleaned/final_object_no_peyers.h5ad'))

In [10]:
adata.obs['Subtype'] = adata.obs['Subtype'].replace('Cd8_T-Cell_P14', 'Cd8_T-Cell_ab+')

In [11]:
adata.write(os.path.join(output_folder, 'analysis/cleaned/final_object_no_peyers.h5ad'))